In [ ]:
#Installing needed libraries
!pip install geopandas

In [ ]:
#Importing needed libraries
import pandas as pd
import geopandas as gpd

In [ ]:
#Loading CEQR projects data 
ceqr_projects = pd.read_csv('CEQR_Projects.csv')
ceqr_projects.head()

,CEQR,Project Name,Project Description,Borough,Lead Agency,URL
0,04DCP052Q,Brookville Rezoning,To amend the Zoning Map to rezone 88 blocks fr...,Queens,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...
1,93TLC001Y,Community Cab Proposal,Proposal for a new classification of For-Hire ...,Citywide,Taxi and Limousine Commission,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...
2,99BSA079K,874 Bedford Avenue,Proposed Residential Dwellings in an M1-1 Zone,Brooklyn,Board of Standards and Appeals,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...
3,16DCP043R,22 Norman Place,"The Applicant, 22 Norman Place LLC, is request...",Staten Island,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...
4,94DBS006M,Downtown Lower Manhattan BID,Development of a Business Improvement District,Manhattan,Department of Small Business Services,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...


In [ ]:
#Loading Projects that are only rezoning projects
ceqr_projects = ceqr_projects[ceqr_projects['Project Name'].str.contains("Rezoning")]

In [ ]:
#Loading all project locations
ceqr_locations = pd.read_csv('CEQR_Project_Locations.csv')
ceqr_locations.head()

,CEQR,Project Name,Block,Lot,Community District,House Number,Street Name,Postcode
0,82-270M,46-50 Ganesvoort Street Project,643.0,43.0,MN03,NaN,NaN,NaN
1,82-270M,46-50 Ganesvoort Street Project,643.0,43.0,MN02,NaN,NaN,NaN
2,82-270M,46-50 Ganesvoort Street Project,643.0,49.0,MN03,NaN,NaN,NaN
3,82-270M,46-50 Ganesvoort Street Project,643.0,49.0,MN02,NaN,NaN,NaN
4,82-270M,46-50 Ganesvoort Street Project,643.0,54.0,MN03,NaN,46-50 GANESVOORT STREET,NaN


In [ ]:
#Grouping all addresses as per CEQR number and retaining only first block and lot information 
ceqr_locations=ceqr_locations.groupby('CEQR').first()
ceqr_locations.head()

,Project Name,Block,Lot,Community District,House Number,Street Name,Postcode
CEQR,,,,,,,
00BOE001M,Agency Facility,1077.0,43.0,MN04,NaN,NaN,NaN
00BSA001K,Restaurant on 8th Avenue,1112.0,52.0,BK07,NaN,"1638 8TH AVENUE, WEST 100-0 NORT",NaN
00BSA002M,286 Spring Street,579.0,5.0,MN02,286,SPRING STREET,10013.0
00BSA004M,625 Greenwich Street,602.0,56.0,MN02,625,GREENWICH STREET,10014.0
00BSA005M,East Houston Street Variance,412.0,53.0,MN03,215,EAST HOUSTON STREET,10002.0


In [ ]:
#Loading project milestones datasets
ceqr_milestones = pd.read_csv('CEQR_Project_Milestones.csv')
ceqr_milestones.head()

,CEQR,Project Name,Milestone Name,Milestone Date
0,09DCP036M,246 Eleventh Avenue,Negative Declaration,05/04/2009
1,94BSA034Q,Lucille Roberts Fitness Center,EAS,NaN
2,94BSA037Q,Lucille Roberts Fitness Center,Revised EAS,07/06/1994
3,09DCP048M,Fountain House,Lead Agency Letter,03/31/2009
4,13BSA002K,910 Manhattan Avenue,Negative Declaration,01/08/2013


In [ ]:
#Converting to datetime
ceqr_milestones['Milestone Date']= pd.to_datetime(ceqr_milestones['Milestone Date'])

In [ ]:
#Filtering to take only the last entered stage for the project
ceqr_milestones=ceqr_milestones.groupby('CEQR').last()

In [ ]:
#Joining all CEQR related datasets together
ceqr_data = ceqr_projects.merge(ceqr_locations,how='left',on='CEQR')
ceqr_data = ceqr_data.merge(ceqr_milestones,how='left',on='CEQR')
ceqr_data = ceqr_data.drop(columns=['Project Name_y','Project Name_x'])
ceqr_data.head()

,CEQR,Project Description,Borough,Lead Agency,URL,Block,Lot,Community District,House Number,Street Name,Postcode,Project Name,Milestone Name,Milestone Date
0,04DCP052Q,To amend the Zoning Map to rezone 88 blocks fr...,Queens,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,NaN,NaN,QN13,NaN,"PORTIONS OF 84 BLOCKS, GENERALLY",NaN,Brookville Rezoning,EAS,2004-05-05
1,06DCP020X,Zoning map amendment from R4 to R4A for a 3 bl...,Bronx,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,5559.0,157.0,BX10,NaN,NaN,NaN,Brush Avenue Rezoning,Negative Declaration,2005-09-12
2,06DCP091Q,To rezone all portions of 21 blocks in the wes...,Queens,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,NaN,NaN,NaN,NaN,NaN,NaN,Middle Village Follow Up Rezoning,EAS,2006-03-31
3,09DCP077Q,An amendment to the Zoning Map to extend a C1-...,Queens,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,12775.0,17.0,QN13,NaN,NaN,NaN,Springfield Boulevard Rezoning,Lead Agency Letter,2009-06-11
4,01DCP076X,A zoning map change that would remove an aband...,Bronx,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,3943.0,199.0,BX09,NaN,PURDY STREET,NaN,1701 Purdy Street Rezoning,EAS,2005-01-25


In [ ]:
#Loading BBL shapefiles
bbl = gpd.read_file('mappedpluto.shp')
bbl.head()

,Borough,Block,Lot,ZipCode,SchoolDist,Address,NumFloors,NumBldgs,UnitsTotal,BBL,Latitude,Longitude,Shape_Area,geometry
0,MN,1,10,10004,02,1 GOVERNORS ISLAND,0.0,134,0,1.000010e+09,40.688799,-74.018675,7.550339e+06,"POLYGON ((978037.649 188629.852, 977583.417 18..."
1,MN,871,82,10003,02,34 UNION SQUARE,2.0,1,1,1.008710e+09,40.735733,-73.989153,3.652481e+03,"POLYGON ((987207.112 207368.164, 987319.197 20..."
2,MN,872,1,10003,02,36 UNION SQUARE,7.0,1,76,1.008720e+09,40.736013,-73.988919,1.711478e+04,"POLYGON ((987394.096 207441.437, 987415.986 20..."
3,MN,872,13,10003,02,35 IRVING PLACE,12.0,1,1,1.008720e+09,40.735683,-73.988092,2.570025e+04,"POLYGON ((987616.641 207347.562, 987626.708 20..."
4,MN,872,35,10003,02,166 3 AVENUE,19.0,1,236,1.008720e+09,40.735030,-73.986328,1.835249e+04,"POLYGON ((987953.017 207015.312, 987999.954 20..."


In [ ]:
bbl.dtypes

Borough         object
Block            int64
Lot              int64
ZipCode          int64
SchoolDist      object
Address         object
NumFloors      float64
NumBldgs         int64
UnitsTotal       int64
BBL            float64
Latitude       float64
Longitude      float64
Shape_Area     float64
geometry      geometry
dtype: object

In [ ]:
#Merging BBL and CEQR data with Block and Lot Information
ceqr_bbl = pd.merge(bbl, ceqr_data, on=['Block','Lot'], how='inner', suffixes=['_and', '_or'])
ceqr_bbl.head()

,Borough_and,Block,Lot,ZipCode,SchoolDist,Address,NumFloors,NumBldgs,UnitsTotal,BBL,Latitude,Longitude,Shape_Area,geometry,CEQR,Project Description,Borough_or,Lead Agency,URL,Community District,House Number,Street Name,Postcode,Project Name,Milestone Name,Milestone Date
0,MN,1041,1,10019,02,742 9 AVENUE,1.0,1,1,1.010410e+09,40.763455,-73.988564,2784.750884,"POLYGON ((987455.958 217387.985, 987367.323 21...",03DCP030K,An application by DCP to amend the City zoning...,Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK06,NaN,110 BLOCKS IN PARK SLOPE,NaN,Park Slope Rezoning,EAS,2002-11-15
1,BK,1041,1,11215,15,535 4 AVENUE,12.0,1,154,3.010410e+09,40.666894,-73.991158,19765.778775,"POLYGON ((986802.911 182300.792, 986745.825 18...",03DCP030K,An application by DCP to amend the City zoning...,Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK06,NaN,110 BLOCKS IN PARK SLOPE,NaN,Park Slope Rezoning,EAS,2002-11-15
2,MN,1081,7501,10019,02,525 WEST 52 STREET,24.0,1,395,1.010818e+09,40.766836,-73.991971,38759.587082,"POLYGON ((986605.437 218689.328, 986505.846 21...",13HPD106M,"The proposal involves an application by HPD, o...",Manhattan,Department of Housing Preservation & Development,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,MN04,NaN,NaN,NaN,525 West 52nd Street / 540 West 53rd Street Re...,EAS,2013-11-25
3,BK,1081,7501,11215,15,630 3 STREET,4.0,1,2,3.010818e+09,40.668313,-73.974583,2051.041959,"POLYGON ((991338.131 182800.689, 991282.346 18...",13HPD106M,"The proposal involves an application by HPD, o...",Manhattan,Department of Housing Preservation & Development,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,MN04,NaN,NaN,NaN,525 West 52nd Street / 540 West 53rd Street Re...,EAS,2013-11-25
4,MN,42,1,10005,02,100 MAIDEN LANE,24.0,1,346,1.000420e+09,40.706718,-74.007221,19536.749132,"POLYGON ((982344.744 196757.500, 982267.019 19...",01DCP026K,Zoning map amendment to rezone an area from M1...,Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK02,NaN,BLOCKFRONTS ON THE EAST SIDE OF,NaN,Bridge and Water Streets Rezoning,Negative Declaration,2003-12-15


In [ ]:
#Installing necessary geospatial libraries
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
!pip install pyspark
!pip install fiona
!pip install shapely

In [ ]:
#Importing needed libraries
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

import seaborn as sns
sns.set(style="whitegrid")

import fiona
import fiona.crs
import shapely
import rtree

import pandas as pd
import geopandas as gpd

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#Loading census tract information
nyc_counties = ["005","061","081","085","047"]
spatial_data = gpd.read_file('tl_2019_36_bg.shp')
spatial_data = spatial_data.loc[spatial_data['COUNTYFP'].isin(nyc_counties)]
spatial_data.to_file("block_group.json", driver="GeoJSON")

In [ ]:
#ceqr_bbl.to_csv('ceqr_bbl.csv')
#ceqr_bbl.to_file("ceqr_bbl.json", driver="GeoJSON")

In [ ]:
#Converting to GeoJSON 

#bbls = gpd.read_file('ceqr_bbl.json').to_crs(fiona.crs.from_epsg(2263))
block_groups = gpd.read_file('block_group.json').to_crs(fiona.crs.from_epsg(2263))

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
#Creating rtree indexes for spatial matching of BBLs and Census Tracts
index = rtree.Rtree()
for idx,geometry in enumerate(block_groups.geometry):
    index.insert(idx, geometry.bounds)

In [ ]:
index.bounds

[912190.8893736254, 113288.79017251907, 1067382.526638073, 273587.03900878486]

In [ ]:
idx

6492

In [ ]:
#Spatial matching of BBLs and Census Tracts

import csv
import pyproj
import shapely.geometry as geom


proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    

counts = []

for i, row in ceqr_bbl.iterrows():
  p = geom.Point(proj(float(row['Longitude']), float(row['Latitude'])))
  match = None
  for idx in index.intersection((p.x, p.y, p.x, p.y)):
    if block_groups.geometry[idx].contains(p):
      match = block_groups.GEOID[idx]
      counts.append(match)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))


In [ ]:
#Creating new column with Census Tract GEOIDs
ceqr_bbl['GEOID'] = counts
ceqr_bbl.head()

,Unnamed: 0,Borough_and,Block,Lot,ZipCode,SchoolDist,Address,NumFloors,NumBldgs,UnitsTotal,BBL,Latitude,Longitude,Shape_Area,geometry,CEQR,Project Description,Borough_or,Lead Agency,URL,Community District,House Number,Street Name,Postcode,Project Name,Milestone Name,Milestone Date,GEOID
0,0,MN,1041,1,10019,2,742 9 AVENUE,1.0,1,1,1.010410e+09,40.763455,-73.988564,2784.750884,"POLYGON ((987455.9575104713 217387.9845161438,...",03DCP030K,An application by DCP to amend the City zoning...,Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK06,NaN,110 BLOCKS IN PARK SLOPE,NaN,Park Slope Rezoning,EAS,2002-11-15,360610133002
1,1,BK,1041,1,11215,15,535 4 AVENUE,12.0,1,154,3.010410e+09,40.666894,-73.991158,19765.778775,"POLYGON ((986802.9105706215 182300.7915754318,...",03DCP030K,An application by DCP to amend the City zoning...,Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK06,NaN,110 BLOCKS IN PARK SLOPE,NaN,Park Slope Rezoning,EAS,2002-11-15,360470141003
2,2,MN,1081,7501,10019,2,525 WEST 52 STREET,24.0,1,395,1.010818e+09,40.766836,-73.991971,38759.587082,"POLYGON ((986605.4368753433 218689.3277249336,...",13HPD106M,"The proposal involves an application by HPD, o...",Manhattan,Department of Housing Preservation & Development,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,MN04,NaN,NaN,NaN,525 West 52nd Street / 540 West 53rd Street Re...,EAS,2013-11-25,360610135002
3,3,BK,1081,7501,11215,15,630 3 STREET,4.0,1,2,3.010818e+09,40.668313,-73.974583,2051.041959,"POLYGON ((991338.1305170059 182800.6885209084,...",13HPD106M,"The proposal involves an application by HPD, o...",Manhattan,Department of Housing Preservation & Development,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,MN04,NaN,NaN,NaN,525 West 52nd Street / 540 West 53rd Street Re...,EAS,2013-11-25,360470165003
4,4,MN,42,1,10005,2,100 MAIDEN LANE,24.0,1,346,1.000420e+09,40.706718,-74.007221,19536.749132,"POLYGON ((982344.7436294556 196757.4998474121,...",01DCP026K,Zoning map amendment to rezone an area from M1...,Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK02,NaN,BLOCKFRONTS ON THE EAST SIDE OF,NaN,Bridge and Water Streets Rezoning,Negative Declaration,2003-12-15,360610007001


In [ ]:
#Saving file
ceqr_bbl.to_csv('ceqr_bbl.csv')

In [ ]:
ceqr_bbl = pd.read_csv('ceqr_bbl.csv')

In [ ]:
ceqr_bbl[ceqr_bbl['Milestone Name'] == 'Positive Declaration']['Project Description'].value_counts()

The applicant, Franklin Ave. Acquisition, LLC., is requesting a zoning map change from R6A to R9D/C2-4, an amendment to ZR Appendix F to establish an MIH area, and a Special Permit per ZR Section 74-74 to create a LSGD, to faciliate a new 1,166,027 sf mixed-use development, including 1,135,969 sf residential use (1,450 DU's), 23,922 sf commercial use, and 6,136 sf of community facility use at 960 Franklin Avenue, in Crown Heights, CD 9, Brooklyn.    2
Rezoning From M1-6 to C6-4X                                                                                                                                                                                                                                                                                                                                                                                                                                           2
This is a private application by G4 Capital Partners requesting a zoning map change from

## Getting 5 and 10 year multipliers for price after a CEQR project was approved

In [ ]:
#Getting building sales data to see changes due to CEQR projects
building_sales = pd.read_csv('combined_sales.csv')
building_sales = building_sales.drop(columns=['Unnamed: 0'])
building_sales = building_sales.rename(columns={'ZIP CODE':'ZipCode'})
building_sales.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZipCode,RESIDENTIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,3,PARK SLOPE,22 STORE BUILDINGS,4,964,1,K5,275 4 AVENUE,11215.0,0,2,"10,000","1,500",1995.0,4,K5,0,2005-01-01
1,3,FLATBUSH-CENTRAL,29 COMMERCIAL GARAGES,4,5210,28,G5,1261 FLATBUSH AVENUE,11226.0,0,1,"1,664",900,1933.0,4,G5,0,2005-01-01
2,3,CROWN HEIGHTS,07 RENTALS - WALKUP APARTMENTS,2,1403,7,C7,1722 UNION STREET,11213.0,19,25,"5,300","18,216",1914.0,2,C7,0,2005-01-01
3,3,BOROUGH PARK,12 CONDOS - WALKUP APARTMENTS,2,5703,1402,R2,5813 11TH AVENUE,11219.0,1,1,0,0,0.0,2,R2,242182,2005-01-01
4,3,SHEEPSHEAD BAY,31 COMMERCIAL VACANT LAND,,7363,67,,3437 NOSTRAND AVENUE,11229.0,0,0,0,0,0.0,4,V1,0,2005-01-01


In [ ]:
#Merging CEQR data with building sales data
ceqr_sales = pd.merge(building_sales,ceqr_bbl, on='ZipCode',how='left')
ceqr_sales.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZipCode,RESIDENTIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,Unnamed: 0,Unnamed: 0.1,Borough_and,Block,Lot,SchoolDist,Address,NumFloors,NumBldgs,UnitsTotal,BBL,Latitude,Longitude,Shape_Area,geometry,CEQR,Project Description,Borough_or,Lead Agency,URL,Community District,House Number,Street Name,Postcode,Project Name,Milestone Name,Milestone Date,GEOID
0,3,PARK SLOPE,22 STORE BUILDINGS,4,964,1,K5,275 4 AVENUE,11215.0,0,2,"10,000","1,500",1995.0,4,K5,0,2005-01-01,1.0,1.0,BK,1041.0,1.0,15.0,535 4 AVENUE,12.0,1.0,154.0,3.010410e+09,40.666894,-73.991158,19765.778775,"POLYGON ((986802.9105706215 182300.7915754318,...",03DCP030K,An application by DCP to amend the City zoning...,Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK06,NaN,110 BLOCKS IN PARK SLOPE,NaN,Park Slope Rezoning,EAS,2002-11-15,3.604701e+11
1,3,PARK SLOPE,22 STORE BUILDINGS,4,964,1,K5,275 4 AVENUE,11215.0,0,2,"10,000","1,500",1995.0,4,K5,0,2005-01-01,3.0,3.0,BK,1081.0,7501.0,15.0,630 3 STREET,4.0,1.0,2.0,3.010818e+09,40.668313,-73.974583,2051.041959,"POLYGON ((991338.1305170059 182800.6885209084,...",13HPD106M,"The proposal involves an application by HPD, o...",Manhattan,Department of Housing Preservation & Development,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,MN04,NaN,NaN,NaN,525 West 52nd Street / 540 West 53rd Street Re...,EAS,2013-11-25,3.604702e+11
2,3,PARK SLOPE,22 STORE BUILDINGS,4,964,1,K5,275 4 AVENUE,11215.0,0,2,"10,000","1,500",1995.0,4,K5,0,2005-01-01,14.0,14.0,BK,993.0,55.0,15.0,305 7 STREET,4.0,1.0,8.0,3.009930e+09,40.671019,-73.986085,3191.860654,"POLYGON ((988140.2162885666 183790.8056573868,...",01DCP067M,To rezone the northern portion of a block boun...,Manhattan,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,MN05,1441,BROADWAY,10018.0,1441 Broadway Rezoning,EAS,2001-05-10,3.604701e+11
3,3,PARK SLOPE,22 STORE BUILDINGS,4,964,1,K5,275 4 AVENUE,11215.0,0,2,"10,000","1,500",1995.0,4,K5,0,2005-01-01,59.0,59.0,BK,970.0,1.0,15.0,297 5 AVENUE,4.0,1.0,8.0,3.009700e+09,40.673363,-73.982458,2466.553245,"POLYGON ((989082.0552911758 184637.8141307831,...",06DCP039M,A zoning map amendment to change existing C1-9...,Manhattan,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,MN06,NaN,NaN,NaN,First Avenue Properties Rezoning,Public Hearing on DEIS,2007-12-05,3.604701e+11
4,3,PARK SLOPE,22 STORE BUILDINGS,4,964,1,K5,275 4 AVENUE,11215.0,0,2,"10,000","1,500",1995.0,4,K5,0,2005-01-01,72.0,72.0,BK,1094.0,38.0,15.0,616 10 STREET,2.0,1.0,2.0,3.010940e+09,40.665213,-73.980377,1695.936510,"POLYGON ((989715.4844856262 181679.8328456879,...",11DCP068M,The rezoning proposal is comprised of the foll...,Manhattan,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,MN04,NaN,NaN,NaN,West Clinton Rezoning,EAS,2010-12-29,3.604702e+11


In [ ]:
#Getting sale and project year from the data
ceqr_sales['sale_year'] = pd.DatetimeIndex(ceqr_sales['SALE DATE']).year
ceqr_sales['project_year'] = pd.DatetimeIndex(ceqr_sales['Milestone Date']).year
ceqr_sales.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZipCode,RESIDENTIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,Unnamed: 0,Unnamed: 0.1,Borough_and,Block,Lot,SchoolDist,Address,NumFloors,NumBldgs,UnitsTotal,BBL,Latitude,Longitude,Shape_Area,geometry,CEQR,Project Description,Borough_or,Lead Agency,URL,Community District,House Number,Street Name,Postcode,Project Name,Milestone Name,Milestone Date,GEOID,sale_year,project_year
0,3,PARK SLOPE,22 STORE BUILDINGS,4,964,1,K5,275 4 AVENUE,11215.0,0,2,"10,000","1,500",1995.0,4,K5,0,2005-01-01,1.0,1.0,BK,1041.0,1.0,15.0,535 4 AVENUE,12.0,1.0,154.0,3.010410e+09,40.666894,-73.991158,19765.778775,"POLYGON ((986802.9105706215 182300.7915754318,...",03DCP030K,An application by DCP to amend the City zoning...,Brooklyn,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,BK06,NaN,110 BLOCKS IN PARK SLOPE,NaN,Park Slope Rezoning,EAS,2002-11-15,3.604701e+11,2005,2002.0
1,3,PARK SLOPE,22 STORE BUILDINGS,4,964,1,K5,275 4 AVENUE,11215.0,0,2,"10,000","1,500",1995.0,4,K5,0,2005-01-01,3.0,3.0,BK,1081.0,7501.0,15.0,630 3 STREET,4.0,1.0,2.0,3.010818e+09,40.668313,-73.974583,2051.041959,"POLYGON ((991338.1305170059 182800.6885209084,...",13HPD106M,"The proposal involves an application by HPD, o...",Manhattan,Department of Housing Preservation & Development,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,MN04,NaN,NaN,NaN,525 West 52nd Street / 540 West 53rd Street Re...,EAS,2013-11-25,3.604702e+11,2005,2013.0
2,3,PARK SLOPE,22 STORE BUILDINGS,4,964,1,K5,275 4 AVENUE,11215.0,0,2,"10,000","1,500",1995.0,4,K5,0,2005-01-01,14.0,14.0,BK,993.0,55.0,15.0,305 7 STREET,4.0,1.0,8.0,3.009930e+09,40.671019,-73.986085,3191.860654,"POLYGON ((988140.2162885666 183790.8056573868,...",01DCP067M,To rezone the northern portion of a block boun...,Manhattan,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,MN05,1441,BROADWAY,10018.0,1441 Broadway Rezoning,EAS,2001-05-10,3.604701e+11,2005,2001.0
3,3,PARK SLOPE,22 STORE BUILDINGS,4,964,1,K5,275 4 AVENUE,11215.0,0,2,"10,000","1,500",1995.0,4,K5,0,2005-01-01,59.0,59.0,BK,970.0,1.0,15.0,297 5 AVENUE,4.0,1.0,8.0,3.009700e+09,40.673363,-73.982458,2466.553245,"POLYGON ((989082.0552911758 184637.8141307831,...",06DCP039M,A zoning map amendment to change existing C1-9...,Manhattan,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,MN06,NaN,NaN,NaN,First Avenue Properties Rezoning,Public Hearing on DEIS,2007-12-05,3.604701e+11,2005,2007.0
4,3,PARK SLOPE,22 STORE BUILDINGS,4,964,1,K5,275 4 AVENUE,11215.0,0,2,"10,000","1,500",1995.0,4,K5,0,2005-01-01,72.0,72.0,BK,1094.0,38.0,15.0,616 10 STREET,2.0,1.0,2.0,3.010940e+09,40.665213,-73.980377,1695.936510,"POLYGON ((989715.4844856262 181679.8328456879,...",11DCP068M,The rezoning proposal is comprised of the foll...,Manhattan,Department of City Planning,https://a002-ceqraccess.nyc.gov/ceqr/ProjectIn...,MN04,NaN,NaN,NaN,West Clinton Rezoning,EAS,2010-12-29,3.604702e+11,2005,2010.0


In [ ]:
#Grouping data to display sale price for each year - further grouped under GEOID and project year initiated within each GEOID
ceqr_sale_grouped = pd.DataFrame(ceqr_sales.groupby(['GEOID','project_year','sale_year'])['SALE PRICE'].agg('mean'))
ceqr_sale_grouped.head()

SALE PRICE
GEOID        project_year sale_year            
3.600504e+11 2015.0       2005         407664.0
                          2006         722500.0
                          2007         200800.0
                          2008              0.0
                          2009              0.0

In [ ]:
ceqr_sale_grouped.to_csv('ceqr_sale_grouped.csv')
ceqr_sale_grouped = pd.read_csv('ceqr_sale_grouped.csv')
ceqr_sale_grouped.head()

,GEOID,project_year,sale_year,SALE PRICE
0,3.600504e+11,2015.0,2005,407664.0
1,3.600504e+11,2015.0,2006,722500.0
2,3.600504e+11,2015.0,2007,200800.0
3,3.600504e+11,2015.0,2008,0.0
4,3.600504e+11,2015.0,2009,0.0


In [ ]:
#Adjusting for inflation
inflation_rate_2000 = 1.49
inflation_rate_2001 = 1.45
inflation_rate_2002 = 1.43
inflation_rate_2003 = 1.39
inflation_rate_2004 = 1.36
inflation_rate_2005 = 1.31
inflation_rate_2006 = 1.27
inflation_rate_2007 = 1.24
inflation_rate_2008 = 1.19
inflation_rate_2009 = 1.20
inflation_rate_2010 = 1.18
inflation_rate_2011 = 1.14
inflation_rate_2012 = 1.12
inflation_rate_2013 = 1.10
inflation_rate_2014 = 1.08
inflation_rate_2015 = 1.08
inflation_rate_2016 = 1.07
inflation_rate_2017 = 1.05
inflation_rate_2018 = 1.02
inflation_rate_2019 = 1.00

In [ ]:
#Adjusting for inflation
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2000,'SALE PRICE':]*=inflation_rate_2000
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2001,'SALE PRICE':]*=inflation_rate_2001
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2002,'SALE PRICE':]*=inflation_rate_2002
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2003,'SALE PRICE':]*=inflation_rate_2003
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2004,'SALE PRICE':]*=inflation_rate_2004
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2005,'SALE PRICE':]*=inflation_rate_2005
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2006,'SALE PRICE':]*=inflation_rate_2006
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2007,'SALE PRICE':]*=inflation_rate_2007
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2008,'SALE PRICE':]*=inflation_rate_2008
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2009,'SALE PRICE':]*=inflation_rate_2009
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2010,'SALE PRICE':]*=inflation_rate_2010
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2011,'SALE PRICE':]*=inflation_rate_2011
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2012,'SALE PRICE':]*=inflation_rate_2012
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2013,'SALE PRICE':]*=inflation_rate_2013
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2014,'SALE PRICE':]*=inflation_rate_2014
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2015,'SALE PRICE':]*=inflation_rate_2015
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2016,'SALE PRICE':]*=inflation_rate_2016
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2017,'SALE PRICE':]*=inflation_rate_2017
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2018,'SALE PRICE':]*=inflation_rate_2018
ceqr_sale_grouped.loc[ceqr_sale_grouped.sale_year==2019,'SALE PRICE':]*=inflation_rate_2019

In [ ]:
ceqr_sale_grouped.head()

,GEOID,project_year,sale_year,SALE PRICE
0,3.600504e+11,2015.0,2005,534039.84
1,3.600504e+11,2015.0,2006,917575.00
2,3.600504e+11,2015.0,2007,248992.00
3,3.600504e+11,2015.0,2008,0.00
4,3.600504e+11,2015.0,2009,0.00


In [ ]:
#Getting sale price of buildings 5 years prior to project year and 5 and 10 years post project year
ceqr_sale_grouped['prior_5'] = ceqr_sale_grouped['project_year'] - 5
ceqr_sale_grouped['post_5'] = ceqr_sale_grouped['project_year'] + 5
ceqr_sale_grouped['post_10'] = ceqr_sale_grouped['project_year'] + 10
ceqr_sale_grouped['on_year'] = ceqr_sale_grouped['project_year']
ceqr_sale_grouped['sale_prior5'] = ceqr_sale_grouped.loc[ceqr_sale_grouped['sale_year'] == ceqr_sale_grouped['prior_5'], 'SALE PRICE']
ceqr_sale_grouped['sale_post5'] = ceqr_sale_grouped.loc[ceqr_sale_grouped['sale_year'] == ceqr_sale_grouped['post_5'], 'SALE PRICE']
ceqr_sale_grouped['sale_post10'] = ceqr_sale_grouped.loc[ceqr_sale_grouped['sale_year'] == ceqr_sale_grouped['post_10'], 'SALE PRICE']
ceqr_sale_grouped['sale_on_year'] = ceqr_sale_grouped.loc[ceqr_sale_grouped['sale_year'] == ceqr_sale_grouped['on_year'], 'SALE PRICE']
ceqr_sale_grouped.head()

,GEOID,project_year,sale_year,SALE PRICE,prior_5,post_5,post_10,on_year,sale_prior5,sale_post5,sale_post10,sale_on_year
0,3.600504e+11,2015.0,2005,534039.84,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN
1,3.600504e+11,2015.0,2006,917575.00,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN
2,3.600504e+11,2015.0,2007,248992.00,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN
3,3.600504e+11,2015.0,2008,0.00,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN
4,3.600504e+11,2015.0,2009,0.00,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN


In [ ]:
ceqr_sale_grouped.head(11)

,GEOID,project_year,sale_year,SALE PRICE,prior_5,post_5,post_10,on_year,sale_prior5,sale_post5,sale_post10,sale_on_year
0,3.600504e+11,2015.0,2005,5.340398e+05,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN
1,3.600504e+11,2015.0,2006,9.175750e+05,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN
2,3.600504e+11,2015.0,2007,2.489920e+05,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN
3,3.600504e+11,2015.0,2008,0.000000e+00,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN
4,3.600504e+11,2015.0,2009,0.000000e+00,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN
5,3.600504e+11,2015.0,2010,6.608236e+04,2010.0,2020.0,2025.0,2015.0,66082.36,NaN,NaN,NaN
6,3.600504e+11,2015.0,2011,1.038888e+05,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN
7,3.600504e+11,2015.0,2012,7.030678e+06,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN
8,3.600504e+11,2015.0,2013,2.352168e+05,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN
9,3.600504e+11,2015.0,2014,2.433240e+05,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,NaN


In [ ]:
ceqr_sale_grouped_df = ceqr_sale_grouped[(ceqr_sale_grouped['prior_5'] == ceqr_sale_grouped['sale_year'])|(ceqr_sale_grouped['post_5'] == ceqr_sale_grouped['sale_year'])|(ceqr_sale_grouped['post_10'] == ceqr_sale_grouped['sale_year'])|(ceqr_sale_grouped['on_year'] == ceqr_sale_grouped['sale_year'])]
ceqr_sale_grouped_df.head(10)

,GEOID,project_year,sale_year,SALE PRICE,prior_5,post_5,post_10,on_year,sale_prior5,sale_post5,sale_post10,sale_on_year
5,3.600504e+11,2015.0,2010,6.608236e+04,2010.0,2020.0,2025.0,2015.0,6.608236e+04,NaN,NaN,NaN
10,3.600504e+11,2015.0,2015,5.284286e+05,2010.0,2020.0,2025.0,2015.0,NaN,NaN,NaN,5.284286e+05
23,3.604700e+11,2019.0,2014,1.221751e+06,2014.0,2024.0,2029.0,2019.0,1.221751e+06,NaN,NaN,NaN
28,3.604700e+11,2019.0,2019,2.123018e+06,2014.0,2024.0,2029.0,2019.0,NaN,NaN,NaN,2.123018e+06
30,3.604700e+11,2006.0,2006,1.481500e+06,2001.0,2011.0,2016.0,2006.0,NaN,NaN,NaN,1.481500e+06
35,3.604700e+11,2006.0,2011,1.083695e+06,2001.0,2011.0,2016.0,2006.0,NaN,1.083695e+06,NaN,NaN
40,3.604700e+11,2006.0,2016,2.550215e+06,2001.0,2011.0,2016.0,2006.0,NaN,NaN,2.550215e+06,NaN
45,3.604700e+11,1996.0,2006,1.481500e+06,1991.0,2001.0,2006.0,1996.0,NaN,NaN,1.481500e+06,NaN
60,3.604700e+11,2001.0,2006,1.481500e+06,1996.0,2006.0,2011.0,2001.0,NaN,1.481500e+06,NaN,NaN
65,3.604700e+11,2001.0,2011,1.083695e+06,1996.0,2006.0,2011.0,2001.0,NaN,NaN,1.083695e+06,NaN


In [ ]:
#Grouping per GEOID
ceqr_sale_grouped_df = ceqr_sale_grouped_df.groupby(['GEOID','project_year'])['sale_prior5','sale_post5','sale_post10','sale_on_year'].agg('mean')
ceqr_sale_grouped_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


sale_prior5  ...  sale_on_year
GEOID        project_year                ...              
3.600504e+11 2015.0        6.608236e+04  ...  5.284286e+05
3.604700e+11 2019.0        1.221751e+06  ...  2.123018e+06
             2006.0                 NaN  ...  1.481500e+06
             1996.0                 NaN  ...           NaN
             2001.0                 NaN  ...           NaN

[5 rows x 4 columns]

In [ ]:
ceqr_sale_grouped_df.to_csv('ceqr_sale_grouped_df.csv')

In [ ]:
ceqr_sale_grouped_df = pd.read_csv('ceqr_sale_grouped_df.csv')
ceqr_sale_grouped_df['GEOID'] = ceqr_sale_grouped_df['GEOID'].astype(int)
ceqr_sale_grouped_df.head()

,GEOID,project_year,sale_prior5,sale_post5,sale_post10,sale_on_year
0,360050414004,2015.0,6.608236e+04,NaN,NaN,5.284286e+05
1,360470001001,2019.0,1.221751e+06,NaN,NaN,2.123018e+06
2,360470001002,2006.0,NaN,1.083695e+06,2.550215e+06,1.481500e+06
3,360470009001,1996.0,NaN,NaN,1.481500e+06,NaN
4,360470009002,2001.0,NaN,1.481500e+06,1.083695e+06,NaN


In [ ]:
#Getting percent change within 5 and 10 years after CEQR project has been initiated
ceqr_sale_grouped_df['5yr_perc_growth'] = ((ceqr_sale_grouped_df['sale_post5'] - ceqr_sale_grouped_df['sale_on_year'])/ceqr_sale_grouped_df['sale_on_year'])*100
ceqr_sale_grouped_df['10yr_perc_growth'] = ((ceqr_sale_grouped_df['sale_post10'] - ceqr_sale_grouped_df['sale_on_year'])/ceqr_sale_grouped_df['sale_on_year'])*100
ceqr_sale_grouped_df.head()

,GEOID,project_year,sale_prior5,sale_post5,sale_post10,sale_on_year,5yr_perc_growth,10yr_perc_growth
0,360050414004,2015.0,6.608236e+04,NaN,NaN,5.284286e+05,NaN,NaN
1,360470001001,2019.0,1.221751e+06,NaN,NaN,2.123018e+06,NaN,NaN
2,360470001002,2006.0,NaN,1.083695e+06,2.550215e+06,1.481500e+06,-26.851542,72.137366
3,360470009001,1996.0,NaN,NaN,1.481500e+06,NaN,NaN,NaN
4,360470009002,2001.0,NaN,1.481500e+06,1.083695e+06,NaN,NaN,NaN


In [ ]:
geo_price_multiplier = ceqr_sale_grouped_df[['GEOID','5yr_perc_growth','10yr_perc_growth']]
geo_price_multiplier.to_csv('geo_price_multiplier.csv')

In [ ]:
geo_price_multiplier.shape

(174, 3)

In [ ]:
geo_price_multiplier.head()

,GEOID,5yr_perc_growth,10yr_perc_growth
0,360050414004,NaN,NaN
1,360470001001,NaN,NaN
2,360470001002,-26.851542,72.137366
3,360470009001,NaN,NaN
4,360470009002,NaN,NaN


In [ ]:
#Average multiplier for 5 Years
geo_price_multiplier['5yr_perc_growth'].mean()

25.489449524393212

In [ ]:
#Average multiplier for 10 Years
geo_price_multiplier['10yr_perc_growth'].mean()

75.05966328881522